# Reinforcement Learning mit Tic Tac Toe

In diesem Notebook werden wir 2 Agenten in Tic Tac Toe trainieren, indem sie gegeneinander Spielen und voneinander lernen. Das erlernte Spiel wird in einer Policy gespeichert. 
Anschließend kann diese Policy genutzt werden um die Agenten auch gegen Menschen spielen zu lassen.

In [1]:
import numpy as np
import pickle

Was ist Pickle?

Pickle wird als Python library verwendet um am Ende unsere Policy darin zu speichern. Ohne Pickle müsste der Agent jedes mal erneut Trainiert werden, wenn das Programm gestartet wird.

![Reinforcement Learning](RL.png) 

Um Tic Tac Toe umzusetzen benötigen wir erstmal unser Environment, einen Agenten, Aktionen die der Agent in dem Environment ausführen kann und Rewards für die Aktionen. 

Unser Environment besteht einfach aus einem 3x3 Raster. Es stellt die Rahmenbedingungen dar, in denen der Agent agiert.

In [ ]:
BOARD_ROWS = 3
BOARD_COLS = 3

Wir werden 2 Agenten nutzen, welche Gegeneinander spielen. Also wird eine Spielerklasse erstellt. Für das Spielen der Agenten wird eine Zustandsklasse erstellt, in welcher die Züge und rewards definiert werden.

Wir beginnen mit der Init Funktion für das Spielfeld. Wir erstellen ein leeres Board mit zwei Spielern *p1* und *p2*. Beide Erhalten ein Spielersymbol, welches auf dem Board gesetzt wird, sobald einer der Spieler eine Aktion ausführt.

In [3]:
def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1

Die **getHash** Funktion hasht den aktuellen Zustand des Spielbretts, sodass es in einer State-value Bibliothek gespeichert werden kann. Diese State-Value Bibliothek spielt eine wichtige Rolle in der Entscheidungsfindung der Agenten. 

In [4]:
def getHash(self):
    self.boardHash = str(self.board.reshape(BOARD_COLS * BOARD_ROWS))
    return self.boardHash

Sobald ein Spieler eine Aktion ausgeführt hat, wird das Spielersymbol auf dem Spielbrett eingetragen und die Verfügbaren Positionen geupdated. Außerdem wird der Spieler gewechselt.

In [5]:
def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions

def updateState(self, position):
    self.board[position] = self.playerSymbol
    # switch to another player
    self.playerSymbol = -1 if self.playerSymbol == 1 else 1

Nach jeder Aktion wird geprüft ob bereits ein Gewinner feststeht. Falls ein Gewinner feststeht muss außerdem ein Reward verteilt werden. Die Funktion prüft zuerst ob in einer Reihe oder Zeile 3 Spieler Symbole liegen und anschließend in der Diagonalen. 

Bei Beendung des Spiels werden folgende Rewards verteilt:

- 1, wenn gewonnen
- 0, wenn verloren
- 0.1 für *p1*, wenn Unentschieden (Unentschieden soll vermieden werden) 
- 0.5 für *p2*

In [6]:
def winner(self):
        # row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # col
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])
        diag_sum = max(abs(diag_sum1), abs(diag_sum2))
        if diag_sum == 3:
            self.isEnd = True
            if diag_sum1 == 3 or diag_sum2 == 3:
                return 1
            else:
                return -1

        # tie
        # no available positions
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
        # not end
        self.isEnd = False
        return None

# only when game ends
def giveReward(self):
        result = self.winner()
        # backpropagate reward
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.1)
            self.p2.feedReward(0.5)

# Spielerklasse

Wir benötigen nun eine Spielerklasse die unseren Agenten darstellt. Der Agent muss für folgendes in der Lage sein:

1. Aktionen auswählen basierend auf den bekannten Zuständen 
2. Alle Zustände des Spiels aufzeichnen 
3. Seine States-Value Schätzung nach jedem Spiel updaten
4. Seine Policy speichern und laden

In der Liste *self.states* halten wir alle Züge fest, die während des Spiels gemacht wurden.  
Wir legen ein *dict* für die State-Value paare an und updaten die Schätzung am Ende jedes Spiels.  
Die Learning Rate (self.lr) bestimmt, wie stark die Gewichtung von neuen Informationen gegüber bereits vorhandenen Informationen ist, wenn der Reward für einen bestimmten Zustand Aktuallisiert wird. Die LR spielt am ende jedes Spiels eine Rolle, wenn wir unsere State-Values Aktualisieren.  
Wir nutzen den ϵ-greedy um eine Balancierung zwischen Exploration und Exploitation zu nutzen (0.3). Das Bedeutet 70% der Zeit sind wir greedy und wählen unsere Aktion basierend auf unserer aktuellen Schätzung aus und 30% der Zeit wählen wir einen zufälligen Schritt.  
Das Decay Gamma bestimmt, wie wichtig zukünftige Belohnungen im Vergleich zu unmittelbaren Belohnungen sind. Bei einer 1 würden wir zukünftige Belohnungen genauso hoch gewichten wie unmittelbare Belohnungen, was einer langfristige Sichweise entspricht. Bei 0 würden wir nur unmittelbare Belohnungen berücksichtigen (kurzfristige Sichtweise)

In [7]:
def __init__(self, name, exp_rate=0.3):
        self.name = name
        self.states = []  # record all positions taken
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.9
        self.states_value = {}  # state -> value

Als nächstes wird die **chooseAction** Methode implementiert. Hier nutzen wir unseren Greedy Faktor um zuerst zu bestimmen ob wir eine zufällige Position wählen oder unser State-value dict nutzen. Falls wir unser State-Value dict nutzen, erstellen wir nun für alle Möglichen positionen ein neues Board setzen unser Spielersymbol. So können wir einmal durch alle Möglichen Positionen durch iterieren und wählen den Zug, mit der dem höchsten State-value. 

![image.png](State-value.png)

In [8]:
def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate:
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                value = 0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                # print("value", value)
                if value >= value_max:
                    value_max = value
                    action = p
        # print("{} takes action {}".format(self.name, action))
        return action

Nun kommt der wichtigste Teil. Die Rückpropagierung. Für die Rückpropagierung wird folgende Formel verwendet:

![image.png](Formel.png)

Die Formel muss nicht genau verstanden werden, wichtig ist nur, dass es sich um sogenannte Werteiteration handelt. Die Logik dahinter ist, dass wir den aktuellen Wert langsam auf Basis unserer neuesten Beobachtung anpassen. Alle unsere Positionen und Züge werden während des Spiels in der Liste self.States gespeichert. Wenn der Agent das Ende des Spiels erreicht, werden die Schätzwerte in umgekehrter Reihenfolge aktualisiert. So ergeben sich die State-Values und der Agent wird besser im Spiel.

In [10]:
def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr * (self.decay_gamma * reward - self.states_value[st])
            reward = self.states_value[st]

# Training

Nun ist der Agent in der Lage zu lernen und alles ist aufgesetzt. Wir lassen nun beide Spieler gegeneinander Spielen um gegenseitig voneinander zu lernen.  
  
Während des Trainings durchgeht jeder Agent für jeden Zug folgenden Prozess:
- Prüft welche offenen Positionen es gibt
- Wählt eine Aktion/Handlung aus
- Updatet den Zustand des Spielbretts und fügt seinen Spielzug der Liste self.states hinzu
- Prüft ob das Spiel vorbei ist und verteilt dementsprechend rewards

In [11]:
def play(self, rounds=100):
        for i in range(rounds):
            if i % 1000 == 0:
                print("Rounds {}".format(i))
            while not self.isEnd:
                # Player 1
                positions = self.availablePositions()
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                # take action and upate board state
                self.updateState(p1_action)
                board_hash = self.getHash()
                self.p1.addState(board_hash)
                # check board status if it is end

                win = self.winner()
                if win is not None:
                    # self.showBoard()
                    # ended with p1 either win or draw
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break

                else:
                    # Player 2
                    positions = self.availablePositions()
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                    self.updateState(p2_action)
                    board_hash = self.getHash()
                    self.p2.addState(board_hash)

                    win = self.winner()
                    if win is not None:
                        # self.showBoard()
                        # ended with p2 either win or draw
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break

# Policy Speichern

Am Ende muss nur noch die Policy gespeichert werden, sodass wir diese Laden können um gegen einen Menschen zu spielen. 

In [12]:
def savePolicy(self):
        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

def loadPolicy(self, file):
        fr = open(file, 'rb')
        self.states_value = pickle.load(fr)
        fr.close()

# Mensch gegen Computer

Wir erstellen einen Menschklasse, welche dann gegen unsere Agenten spielen kann.